In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
fonte_texto = dict(
        family="Arial, serif",  # Fonte gótica
        size=14,                              # Tamanho da fonte
        color="white"                         # Cor da fonte
    )

fonte_titulo = dict(
            size=24,              # Tamanho do título
            family="Old English Text MT, serif",  # Fonte do título
            color="white"      # Cor do título
        )

legenda = dict(
        title="Legenda",
        x=1,
        y=1,
        bgcolor="black",
        bordercolor="white",
        borderwidth=1
)

# Limpeza

In [4]:
base = pd.read_csv("relatorio_streamlit.csv")

In [5]:
# Corrigir a definição dos nomes das colunas
base.columns = ['id', 'cliente', 'etapa', 'origem', 'convenio', 'comissao_total', 
                'valor_pago', 'tipo_campanha', 'tag_campanha', 'telefone', 'cpf', 
                'vendedor_proprietario']

# Aplicar a substituição na coluna 'etapa' usando raw string para evitar SyntaxWarning
base['etapa'] = base['etapa'].astype(str).str.replace(r" \( Pipeline de Vendas\)", "", regex=True)
base['etapa'] = base['etapa'].astype(str).str.replace(r" \(Pipeline de Disparos?\)", "", regex=True).str.upper()


# Remover ".0" da coluna 'cpf'
base['cpf'] = base['cpf'].astype(str).str.replace(".0", "")

In [6]:
bins = [0, 499, 800, 1200, 1500, float('inf')]
labels = ['0-499', '500-800', '800-1200', '1200-1500', '1800+']
base['faixa_comissao'] = pd.cut(base['comissao_total'], bins=bins, labels=labels, right=True, include_lowest=True)

# 1 - Quantidade de leads por origem

In [ ]:
leads_por_origem = base.groupby("origem")['id'].size().reset_index(name="quantidade").sort_values(by='quantidade', ascending=False)
leads_por_origem = leads_por_origem.loc[leads_por_origem['quantidade'] > 10]

In [ ]:
fig = px.bar(leads_por_origem,
              x='quantidade',
              y='origem',
              color='origem',
              barmode='group'
             )

fig.update_layout(title=dict(
                        text="Quantidade de leads gerados",
                        x=0.5,                                # Centraliza o título
                        xanchor='center',
                        yanchor='top', 
                        font=fonte_titulo), 
                  xaxis_title="Quantidade", 
                  yaxis_title="",
                  font=fonte_texto,
                  width=1080,
                  height=500,
                  template='plotly_dark',
                  margin=dict(l=10, r=0, t=50, b=50),
                  legend=legenda,
                  plot_bgcolor='black',
                  paper_bgcolor='black'
                 )

fig.show()

# 2 - Distribuição dos leads no funil por origem

In [ ]:
distrib_funil_por_origem = base.groupby(['origem', 'etapa'])['id'].size().reset_index(name='quantidade')
totais_por_origem = distrib_funil_por_origem.groupby('origem')['quantidade'].sum()
origens_validas = totais_por_origem[totais_por_origem > 5].index
distrib_funil_por_origem_filtrado = distrib_funil_por_origem[distrib_funil_por_origem['origem'].isin(origens_validas)]

In [ ]:
fig = px.bar(
    distrib_funil_por_origem_filtrado, 
    x='origem',               # Agrupamento no eixo X por origem
    y='quantidade',           # Quantidade no eixo Y
    color='etapa',            # Diferenciar pelas etapas (barra agrupada por cor)
    barmode='group',          # Configuração para barras agrupadas
    title="Distribuição das Etapas por Origem",  # Título do gráfico
    labels={
        'origem': 'Origem dos Leads',
        'quantidade': 'Quantidade',
        'etapa': 'Etapas do Funil'
    }
)

fig.update_layout(title=dict(
                        text="Distribuição de leads no funil por origem",
                        x=0.5,                                # Centraliza o título
                        xanchor='center',
                        yanchor='top', 
                        font=fonte_titulo), 
                  xaxis_title="Origem", 
                  yaxis_title="Quantidade",
                  font=fonte_texto,
                  width=1080,
                  height=500,
                  template='plotly_dark',
                  margin=dict(l=10, r=0, t=50, b=50),
                  legend=legenda,
                  plot_bgcolor='black',
                  paper_bgcolor='black'
                 )

fig.show()

# 3 - Leads por Convênio Segmentados por Origem

In [ ]:
leads_por_convenio = base.groupby(['convenio', 'origem'])['id'].size().reset_index(name='quantidade')
leads_por_convenio = leads_por_convenio.loc[leads_por_convenio['quantidade'] > 5]

totais_por_convenio = leads_por_convenio.groupby('convenio')['quantidade'].sum().sort_values(ascending=False)
leads_por_convenio = leads_por_convenio.set_index('convenio')  # Define 'convenio' como índice temporariamente
leads_por_convenio = leads_por_convenio.loc[totais_por_convenio.index]  # Reordena pelo índice
leads_por_convenio = leads_por_convenio.reset_index()  # Restaura o índice padrão

In [ ]:
quantidade_maxima_convenios = 10
top_convenios = leads_por_convenio.groupby('convenio')['quantidade'].sum() \
                                  .nlargest(quantidade_maxima_convenios).index
leads_por_convenio_filtrado = leads_por_convenio[leads_por_convenio['convenio'].isin(top_convenios)]

In [ ]:
fig = px.bar(
    leads_por_convenio_filtrado, 
    x='quantidade',               # Quantidade no eixo X
    y='convenio',                 # Convênio no eixo Y
    color='origem',               # Diferenciar pelas origens
    title="Distribuição dos Leads por Convênio e Origem",  # Título do gráfico
    labels={
        'origem': 'Origem dos Leads',
        'quantidade': 'Quantidade',
        'convenio': 'Convênio'
    }
)

# Configurações adicionais do layout
fig.update_layout(
    title=dict(
        text="Distribuição de Leads por Convênio e Origem",
        x=0.5,                     # Centraliza o título
        xanchor='center',
        yanchor='top', 
        font=fonte_titulo
    ), 
    xaxis_title="Quantidade",
    yaxis_title="Convênios",
    font=fonte_texto,
    width=1080,
    height=500,
    template='plotly_dark',
    margin=dict(l=10, r=0, t=50, b=50),
    legend=legenda,
    plot_bgcolor='black',
    paper_bgcolor='black'
)

# Mostrar o gráfico
fig.show()


# 4 - Quantidade de leads por convenio segmentado pela comissão

In [ ]:
leads_comissao = base.groupby(['convenio', 'faixa_comissao'], observed=True)['id'].size().reset_index(name='quantidade')

In [ ]:
quantidade_maxima_convenios = 10

# Calcular o total de leads por convênio e selecionar os top N
top_convenios = leads_comissao.groupby('convenio')['quantidade'].sum() \
                              .nlargest(quantidade_maxima_convenios).index

leads_comissao_filtrado = leads_comissao[leads_comissao['convenio'].isin(top_convenios)]

In [ ]:
fig = px.bar(
    leads_comissao_filtrado, 
    x='quantidade',               # Quantidade no eixo X
    y='convenio',                 # Convênio no eixo Y
    color='faixa_comissao',       # Diferenciar pelas faixas de comissão
    title="Quantidade de leads por convênio segmentado pela comissão",  # Título do gráfico
    labels={
        'faixa_comissao': 'Faixa de Comissão',
        'quantidade': 'Quantidade',
        'convenio': 'Convênio'
    }
)

# Configurações adicionais do layout
fig.update_layout(
    title=dict(
        text="Distribuição de Leads por Convênio e Comissão",
        x=0.5,                     # Centraliza o título
        xanchor='center',
        yanchor='top', 
        font=fonte_titulo
    ), 
    xaxis_title="Quantidade",
    yaxis_title="Convênios",
    font=fonte_texto,
    width=1080,
    height=500,
    template='plotly_dark',
    margin=dict(l=10, r=0, t=50, b=50),
    legend=legenda,
    plot_bgcolor='black',
    paper_bgcolor='black'
)

# Mostrar o gráfico
fig.show()

In [12]:
px.colors.sequential.Plasma


['#0d0887',
 '#46039f',
 '#7201a8',
 '#9c179e',
 '#bd3786',
 '#d8576b',
 '#ed7953',
 '#fb9f3a',
 '#fdca26',
 '#f0f921']